[![Open notebook in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/afondiel/computer-science-notebook/tree/master/core/systems/edge-computing/edge-ai/lab/examples/deploy-with-qualcomm/notebooks/intro_to_qai_hub_profiling_models.ipynb)

## Qualcomm AI Hub Stack

![](https://aihub.qualcomm.com/_next/static/media/how-it-works.a071b789.svg)

### Model Profiling Workflow
1. Load Pre-trained Models
2. Trace Model
3. **Compile** Model (unique job id and link for viewing in the portal)
4. **Profile** on cloud-hosted device
5. Download Optimized model

In [ ]:
# prompt: create the code to install torch ecosystems lib
!pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# Set up your Qualcomm AI Hub development environment:
# !pip install -q qai-hub
!pip install qai-hub
!pip install qai_hub_models

In [ ]:
import qai-hub as hub
import torch
from torchvision.models import mobilenet_v2
import requests
import numpy as np
from PIL import Image

In [ ]:
# Access API Token from Google Colab secret keys
ai_hub_api_token = userdata.get('QAI_HUB_API_KEY')
print(f"API Key: {ai_hub_api_token}")

In [ ]:
# Access API using utils module
# from utils import get_ai_hub_api_token
# ai_hub_api_token = get_ai_hub_api_token()

# !qai-hub configure --api_token $ai_hub_api_token

In [ ]:
# Configure the API KEY
!qai-hub configure --api_token $ai_hub_api_token

In [ ]:
# Check if API KEY is valid
!qai-hub list-devices

In [ ]:
# Using python commands
hub.get_devices()

In [ ]:
# Using pre-trained MobileNet
torch_model = mobilenet_v2(pretrained=True)
torch_model.eval()

# Step 1: Trace model
input_shape  = (1, 3, 224, 224) # ImageNet Format
example_input = torch.rand(input_shape)
traced_torch_model = torch.jit.trace(torch_model, example_input)

# Step 2: Compile model
compile_job = hub.submit_compile_job(
  model  = traced_torch_model,
  device = hub.Device("RB3 Gen 2 (Proxy)"),
  input_specs = dict(image=input_shape),
)

# Step 3: Profile on cloud-hosted device
target_model = compile_job.get_target_model()
profile_job = hub.submit_profiie_job(
  model=target_model,
  device=hub.Device("RB3 Gen 2 (Proxy)"),
)
# step 4: Download optimized model
target_model = compile_job.get_target_model()
target_model.download("mobilenet_v2.tflite")

## References
- Qualcomm AI Hub docs:
  - https://aihub.qualcomm.com/
  - https://aihub.qualcomm.com/get-started
  - https://aihub.qualcomm.com/models
  - https://app.aihub.qualcomm.com/docs/
  - https://www.qualcomm.com/developer/artificial-intelligence#overview
  
- Qualcomm Device Cloud:
  - https://qdc.qualcomm.com/
  - https://qdc.qualcomm.com/support/user-guide
- AIMET:
  - Github repo: https://github.com/quic/aimet
  - Documentation (latest release): https://quic.github.io/aimet-pages/releases/latest/
  - Qualcomm Developer: https://www.qualcomm.com/developer/software/ai-model-efficiency-toolkit

- Qualcomm Developer: https://www.qualcomm.com/developer/software/

- Tutorial (YT/Qualcomm Developer): https://youtu.be/8eZEVWWM2YM?si=jAuNm7BMZZTdzMRB